## Overview ##

PubChem is a site run by the NIH which hosts raw data associated with chemical experiments; here we analyze the data hosted at PubChem for assay 1030, which looks for inhibitors of the protein encoding gene ALDH1A1. You can access the page for this assay [here](https://pubchem.ncbi.nlm.nih.gov/bioassay/1030)

## Results ##

We use the SMILES string, a common representation for a molecule amongst chemists, to begin the featurization process. Because the length of this string varies, it is normalized in the form of a Morgan Fingerprint; these are then used to train various binary classifiers

In [1]:
import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.ensemble import RandomForestClassifier

import warnings
warnings.filterwarnings('ignore')

global_random_state = 42

np.random.seed(global_random_state)

In [2]:
# Load assay info. Note: This CSV was obtained from PubChem bioassay aka PCBA, via searching for AID 1030 
# and downloading the datatable

ba_df = pd.read_csv("AID_1030_datatable_all.csv")

# Load compound info
cs_df = pd.read_csv("AID_1030_compound_smiles.csv",sep='\t',header=0)

# Merge the two
full_df = ba_df.merge(cs_df,on='PUBCHEM_CID')

# Cleanup the compound ID column
full_df["PUBCHEM_CID"] = full_df["PUBCHEM_CID"].astype(int)

compound_ids = list()
smiles_list = list()
fingerprints = list()
activities = list()

for index, row in full_df.iterrows() :
    cid = row["PUBCHEM_CID"]
    smiles_string = row["Smiles"]
    mol = Chem.MolFromSmiles(smiles_string)
    is_active = row["PUBCHEM_ACTIVITY_OUTCOME"] == "Active"
    if mol is not None: 
        fingerprint = rdMolDescriptors.GetMorganFingerprintAsBitVect(mol,2,nBits=2048,useChirality=False,
                                                                     useBondTypes=False,useFeatures=False)
        
        # From RDKit documentation
        arr = np.zeros((1,))
        DataStructs.ConvertToNumpyArray(fingerprint, arr)
        fingerprint = arr
        
        compound_ids.append(cid)
        smiles_list.append(smiles_string)
        fingerprints.append(fingerprint)
        activities.append(is_active)
    
    if index % 10000 == 0:
        print("Processed index: {0}".format(index))

print("Processed all, pickling")

compound_ids_and_features = (compound_ids, smiles_list, fingerprints, activities)

# Pickle the data to save time in the future
with open('data.pickle', 'wb') as f:
    pickle.dump(compound_ids_and_features, f, pickle.HIGHEST_PROTOCOL)

Processed index: 0
Processed index: 10000
Processed index: 20000
Processed index: 30000
Processed index: 40000
Processed index: 50000
Processed index: 60000
Processed index: 70000
Processed index: 80000
Processed index: 90000
Processed index: 100000
Processed index: 110000
Processed index: 120000
Processed index: 130000
Processed index: 140000
Processed index: 150000
Processed index: 160000
Processed index: 170000
Processed index: 180000
Processed index: 190000
Processed index: 200000
Processed index: 210000
Processed index: 220000
Processed all, pickling


In [ ]:
# Here we see that a naive approach (without balancing data) yields a poor F1-score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report


# Create a train/test split
X_train, X_test, y_train, y_test = train_test_split(fingerprints, activities, test_size=0.33, random_state=global_random_state)

classifier = DecisionTreeClassifier(random_state=global_random_state)

# Note: This may take 5-10 minutes to run
classifier.fit(X_train,y_train)

# We have the 'ground truth' of which of the held-out 33% of the data was really bioactive. But can we predict it?
# If we could, we could predict which compounds might be bioactive without having to actually test them in a lab

score = classifier.score(X_test,y_test)

print("Classifier obtained a mean accuracy score of: {0}".format(score))

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))

# Note: Unfortunately it's not directly comparable to ROC_AUC calculated in MoleculeNet at: https://arxiv.org/pdf/1703.00564.pdf 
# This is because MoleculeNet looks at a different metric (roc_auc) and also a different task (multiclass prediction across 128 bioassays simultaneously vs binary classification here)

In [3]:
# Duplicate imports in case starting from this cell

import pickle
import numpy as np
import pandas as pd
from rdkit import Chem, DataStructs
from rdkit.Chem import AllChem, rdMolDescriptors
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

compound_ids = None
fingerprints = None
activities = None

global_random_state = 42

with open('data.pickle', 'rb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    (compound_ids, smiles_list, fingerprints, activities) = pickle.load(f)

# Print the number of compounds loaded
print("Successfully featurized {0} compounds.".format(len(compound_ids)))

# Show an example of the raw data
print("Sample ID: {0}".format(compound_ids[0]))
print("Sample fingerprint vector: {0}".format(fingerprints[0]))
print("Was it bioactive? Assay returned: {0}".format(activities[0]))

# Correctly balance the classes
from imblearn.under_sampling import RandomUnderSampler
from collections import Counter

rus = RandomUnderSampler(random_state=global_random_state)
X_resampled, y_resampled = rus.fit_sample(fingerprints, activities)

print(sorted(Counter(y_resampled).items()))

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.33, random_state=global_random_state)

Successfully featurized 220364 compounds.
Sample ID: 6603008
Sample fingerprint vector: [ 0.  0.  0. ...,  0.  0.  0.]
Was it bioactive? Assay returned: False
[(False, 16111), (True, 16111)]


In [6]:
# Create a new train/test split based on resampled data

classifier = DecisionTreeClassifier(random_state=42)

# Note: This may take 5-10 minutes to run
classifier.fit(X_train,y_train)

# We have the 'ground truth' of which of the held-out 33% of the data was really bioactive. But can we predict it?
# If we could, we could predict which compounds might be bioactive without having to actually test them in a lab

score = classifier.score(X_test,y_test)

print("Classifier obtained a mean accuracy score of: {0}".format(score))

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))


Classifier obtained a mean accuracy score of: 0.6418092909535452
             precision    recall  f1-score   support

      False       0.64      0.63      0.64      5269
       True       0.64      0.65      0.65      5365

avg / total       0.64      0.64      0.64     10634



In [7]:
# Let's try using a Random forest

classifier = RandomForestClassifier(n_estimators=100, random_state=global_random_state, n_jobs=-1)

# Note: This may take 5-10 minutes to run
classifier.fit(X_train,y_train)

# We have the 'ground truth' of which of the held-out 33% of the data was really bioactive. But can we predict it?
# If we could, we could predict which compounds might be bioactive without having to actually test them in a lab

score = classifier.score(X_test,y_test)

print("Classifier obtained a mean accuracy score of: {0}".format(score))

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))


Classifier obtained a mean accuracy score of: 0.7219296595824714
             precision    recall  f1-score   support

      False       0.71      0.75      0.73      5269
       True       0.74      0.69      0.72      5365

avg / total       0.72      0.72      0.72     10634



In [8]:
# Does an MLP classifier help?
# Answer: Nope, not really

from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier(random_state=global_random_state)

# Note: This may take 5-10 minutes to run
classifier.fit(X_train,y_train)

# We have the 'ground truth' of which of the held-out 33% of the data was really bioactive. But can we predict it?
# If we could, we could predict which compounds might be bioactive without having to actually test them in a lab

score = classifier.score(X_test,y_test)

print("Classifier obtained a mean accuracy score of: {0}".format(score))

y_pred = classifier.predict(X_test)

print(classification_report(y_test, y_pred))



Classifier obtained a mean accuracy score of: 0.6964453639270265
             precision    recall  f1-score   support

      False       0.67      0.75      0.71      5269
       True       0.72      0.65      0.68      5365

avg / total       0.70      0.70      0.70     10634



In [4]:
import keras
print(keras.backend.backend())

Using TensorFlow backend.


tensorflow


In [6]:
# What about a deep neural network?
# Sample code from: https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/

from keras.models import Sequential
from keras.layers import Dense
from keras import metrics

model = Sequential()
model.add(Dense(12, input_dim=2048, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[metrics.binary_accuracy])

y_train_binary = y_train.astype(int)

print(y_train_binary[0:5])

y_test_binary = y_test.astype(int)

model.fit(X_train, y_train_binary, epochs=20, batch_size=1)

score = model.evaluate(X_test, y_test_binary)

print("\n Loss on test set is: %.2f{0}".format(score))

[1 1 1 0 0]
Epoch 1/20
21588/21588 [==============================] - 40s - loss: 0.6304 - binary_accuracy: 0.6431    
Epoch 2/20
21588/21588 [==============================] - 40s - loss: 0.5614 - binary_accuracy: 0.7099    
Epoch 3/20
21588/21588 [==============================] - 40s - loss: 0.5109 - binary_accuracy: 0.7488    
Epoch 4/20
21588/21588 [==============================] - 40s - loss: 0.4687 - binary_accuracy: 0.7775    
Epoch 5/20
21588/21588 [==============================] - 40s - loss: 0.4273 - binary_accuracy: 0.8058    
Epoch 6/20
21588/21588 [==============================] - 40s - loss: 0.3901 - binary_accuracy: 0.8261    
Epoch 7/20
21588/21588 [==============================] - 40s - loss: 0.3583 - binary_accuracy: 0.8442    
Epoch 8/20
21588/21588 [==============================] - 40s - loss: 0.3295 - binary_accuracy: 0.8600    
Epoch 9/20
21588/21588 [==============================] - 40s - loss: 0.3069 - binary_accuracy: 0.8708    
Epoch 10/20
21588/21588 [

In [7]:
y_pred = model.predict_on_batch(X_test)

print(y_test[0:5])
print(y_pred[0:5])

y_pred_binarized = list()

for pred in y_pred:
    if pred >= .5 :
        y_pred_binarized.append(True)
    else :
        y_pred_binarized.append(False)

print(y_pred_binarized[0:5])
print(classification_report(y_test, y_pred_binarized))


[ True False False  True False]
[[ 0.97291088]
 [ 0.25812885]
 [ 0.23556663]
 [ 0.78397679]
 [ 0.57503569]]
[True, False, False, True, True]
             precision    recall  f1-score   support

      False       0.64      0.66      0.65      5269
       True       0.65      0.64      0.65      5365

avg / total       0.65      0.65      0.65     10634



In [8]:
# What about a larger network size?
from keras.models import Sequential
from keras.layers import Dense
from keras import metrics

model = Sequential()
model.add(Dense(2048, input_dim=2048, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[metrics.binary_accuracy])

y_train_binary = y_train.astype(int)

print(y_train_binary[0:5])

y_test_binary = y_test.astype(int)

model.fit(X_train, y_train_binary, epochs=10, batch_size=1)

score = model.evaluate(X_test, y_test_binary)

print("\n Loss on test set is: %.2f{0}".format(score))

[1 1 1 0 0]
Epoch 1/20
21588/21588 [==============================] - 154s - loss: 0.6503 - binary_accuracy: 0.6295   
Epoch 2/20
21588/21588 [==============================] - 154s - loss: 0.5529 - binary_accuracy: 0.7275   
Epoch 3/20
21588/21588 [==============================] - 154s - loss: 0.4236 - binary_accuracy: 0.8148   
Epoch 4/20
21588/21588 [==============================] - 154s - loss: 0.2686 - binary_accuracy: 0.8974   
Epoch 5/20
21588/21588 [==============================] - 154s - loss: 0.1603 - binary_accuracy: 0.9467   
Epoch 6/20
21588/21588 [==============================] - 154s - loss: 0.1031 - binary_accuracy: 0.9710   
Epoch 7/20
21588/21588 [==============================] - 155s - loss: 0.0714 - binary_accuracy: 0.9810   
Epoch 8/20
21588/21588 [==============================] - 155s - loss: 0.0557 - binary_accuracy: 0.9865   
Epoch 9/20
21588/21588 [==============================] - 155s - loss: 0.0487 - binary_accuracy: 0.9892   
Epoch 10/20
21588/21588 [

In [9]:
y_pred = model.predict_on_batch(X_test)

print(y_test[0:5])
print(y_pred[0:5])

y_pred_binarized = list()

for pred in y_pred:
    if pred >= .5 :
        y_pred_binarized.append(True)
    else :
        y_pred_binarized.append(False)

print(y_pred_binarized[0:5])
print(classification_report(y_test, y_pred_binarized))

[ True False False  True False]
[[  1.88671425e-01]
 [  7.83160649e-05]
 [  3.67873199e-08]
 [  5.14224768e-02]
 [  5.37856657e-04]]
[False, False, False, False, False]
             precision    recall  f1-score   support

      False       0.67      0.75      0.71      5269
       True       0.72      0.64      0.68      5365

avg / total       0.70      0.69      0.69     10634



In [ ]:
# The larger NN doesn't seem to have helped much...